# Импорты и базовые настройки

In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


In [ ]:
# Блок 0. Распаковка датасета из zip-архива

import os
import zipfile

# Путь к zip-файлу с датасетом
ZIP_PATH = "/content/fashion-product-images-small.zip"

# Куда распаковываем архив
EXTRACT_ROOT = "/content"

# Папка, которая появится после распаковки
DATASET_DIR_NAME = "fashion-product-images-small"

# Если датасет ещё не распакован — распакуем
target_dir = os.path.join(EXTRACT_ROOT, DATASET_DIR_NAME)
if not os.path.exists(target_dir):
    with zipfile.ZipFile(ZIP_PATH, "r") as zf:
        zf.extractall(EXTRACT_ROOT)
        print("Архив распакован в:", EXTRACT_ROOT)
else:
    print("Датасет уже распакован в:", target_dir)

# Корень датасета, дальше используется в коде
DATA_ROOT = target_dir
print("DATA_ROOT =", DATA_ROOT)


Архив распакован в: /content
DATA_ROOT = /content/fashion-product-images-small


In [ ]:
# Путь к корню датасета
DATA_ROOT = "/content"
CSV_PATH = os.path.join(DATA_ROOT, "styles.csv")
IMAGES_DIR = os.path.join(DATA_ROOT, "images")

df = pd.read_csv(CSV_PATH, on_bad_lines="skip")

# Удалим строки без id или articleType/baseColour
df = df.dropna(subset=["id", "articleType", "baseColour", "productDisplayName"])
df["id"] = df["id"].astype(str)


In [ ]:
# Маппинг articleType -> наши укрупнённые классы
TYPE_MAP = {
    "Tshirts": "tshirt",
    "Shirts": "shirt",
    "Sweatshirts": "hoodie",
    "Sweaters": "sweater",
    "Jackets": "jacket",
    "Jeans": "jeans",
    "Track Pants": "pants",
    "Trousers": "pants",
    "Shorts": "shorts",
    "Casual Shoes": "sneakers",
    "Sports Shoes": "sneakers",
    "Flip Flops": "sandals",
    "Sandals": "sandals",
}

# Оставим только строки, которые встречаются в TYPE_MAP
df["articleType_mapped"] = df["articleType"].map(TYPE_MAP)
df = df.dropna(subset=["articleType_mapped"])

# Список финальных классов типа одежды
type_classes = sorted(df["articleType_mapped"].unique())
type2idx = {t: i for i, t in enumerate(type_classes)}
idx2type = {i: t for t, i in type2idx.items()}
print("Типы вещей:", type_classes)


Типы вещей: ['hoodie', 'jacket', 'jeans', 'pants', 'sandals', 'shirt', 'shorts', 'sneakers', 'sweater', 'tshirt']


In [ ]:
# Маппинг цветов
# Маппинг «мелких» цветов в крупные группы
COLOR_GROUP_MAP = {
    # черный / серый / белый
    "Black":          "black",
    "Charcoal":       "black",

    "Grey":           "grey",
    "Grey Melange":   "grey",

    "White":          "white",
    "Off White":      "white",
    "Cream":          "white",

    # синие
    "Blue":           "blue",
    "Navy Blue":      "blue",
    "Turquoise Blue": "blue",
    "Teal":           "blue",

    # зелёные
    "Green":            "green",
    "Olive":            "green",
    "Sea Green":        "green",
    "Fluorescent Green":"green",
    "Lime Green":       "green",
    "Khaki":            "green",

    # коричнево-бежевые
    "Brown":         "brown",
    "Coffee Brown":  "brown",
    "Mushroom Brown":"brown",
    "Tan":           "brown",
    "Taupe":         "brown",
    "Beige":         "brown",

    # красные
    "Red":      "red",
    "Maroon":   "red",
    "Burgundy": "red",
    "Rust":     "red",

    # розовые
    "Pink":    "pink",
    "Peach":   "pink",
    "Magenta": "pink",

    # фиолетовые
    "Purple":   "purple",
    "Lavender": "purple",
    "Mauve":    "purple",

    # жёлтые / горчичные
    "Yellow":  "yellow",
    "Mustard": "yellow",

    # оранжевые
    "Orange": "orange",
}

# Всё, что не попадает в маппинг (Multi / Gold / Silver / Bronze), будем выбрасывать
def map_color_group(c):
    c = str(c)
    return COLOR_GROUP_MAP.get(c, None)

df["color_group"] = df["baseColour"].apply(map_color_group)

# Выкинем строки с None (Multi, Gold, Silver, Bronze и т.п.)
df = df.dropna(subset=["color_group"])

# Финальные классы цветов после группировки
color_classes = sorted(df["color_group"].unique())
color2idx = {c: i for i, c in enumerate(color_classes)}
idx2color = {i: c for c, i in color2idx.items()}

print("Цветовые группы:", color_classes)



# Простой хелпер для определения "есть принт / нет принта"
def detect_print_flag(name: str) -> int:
    """
    Грубая эвристика: если в названии есть printed/print/graphic/striped/check,
    считаем, что есть принт.
    """
    name = str(name).lower()
    keywords = [
        "printed", "print", "graphic", "logo",
        "striped", "stripe", "check", "checked",
        "pattern", "patterned", "embroidery", "embroidered"
    ]
    return int(any(kw in name for kw in keywords))

df["type_label"] = df["articleType_mapped"].map(type2idx)
df["color_label"] = df["color_group"].map(color2idx).astype("int32")
df["print_label"] = df["productDisplayName"].apply(detect_print_flag)


Цветовые группы: ['black', 'blue', 'brown', 'green', 'grey', 'orange', 'pink', 'purple', 'red', 'white', 'yellow']


In [ ]:
# Путь до изображений: images/{id}.jpg
def build_img_path(pid: str) -> str:
    return os.path.join(IMAGES_DIR, f"{pid}.jpg")

df["img_path"] = df["id"].apply(build_img_path)

# Оставим только те строки, где файл реально существует (на всякий случай)
df = df[df["img_path"].apply(os.path.exists)]

print("Всего примеров после фильтрации:", len(df))

img_paths   = df["img_path"].values
type_labels = df["type_label"].values.astype("int32")
color_labels = df["color_label"].values.astype("int32")
print_labels = df["print_label"].values.astype("float32")


Всего примеров после фильтрации: 19626


In [ ]:
X_train, X_val, type_train, type_val, color_train, color_val, print_train, print_val = train_test_split(
    img_paths,
    type_labels,
    color_labels,
    print_labels,
    test_size=0.2,
    random_state=42,
    stratify=type_labels  # по типу вещи для баланса
)


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def load_and_preprocess(path, t_label, c_label, p_label):
    # читаем файл
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = preprocess_input(img)  # MobileNetV2 препроцессинг [-1,1]

    labels = {
        "type_output": t_label,
        "color_output": c_label,
        "print_output": p_label,
    }
    return img, labels

def make_dataset(paths, t, c, p, training=True):
    ds = tf.data.Dataset.from_tensor_slices((paths, t, c, p))
    if training:
        ds = ds.shuffle(buffer_size=len(paths), reshuffle_each_iteration=True)
    ds = ds.map(load_and_preprocess, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

train_ds = make_dataset(X_train, type_train, color_train, print_train, training=True)
val_ds   = make_dataset(X_val,   type_val,  color_val,  print_val,  training=False)


In [ ]:
# Загружаем предобученный MobileNetV2
backbone = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling="avg"  # глобальный average pooling -> вектор признаков
)

# Сначала заморозим свёрточную часть
backbone.trainable = False

# Многозадачная модель
inputs = backbone.input
x = backbone.output
x = layers.Dropout(0.3)(x)

# Голова 1: тип вещи (многоклассовая)
type_output = layers.Dense(
    len(type_classes),
    activation="softmax",
    name="type_output"
)(x)

# Голова 2: цвет (многоклассовая)
color_output = layers.Dense(
    len(color_classes),
    activation="softmax",
    name="color_output"
)(x)

# Голова 3: принт (бинарная)
print_output = layers.Dense(
    1,
    activation="sigmoid",
    name="print_output"
)(x)

model = models.Model(
    inputs=inputs,
    outputs=[type_output, color_output, print_output],
    name="clothing_multitask_mobilenetv2"
)

model.summary()


Model: "clothing_multitask_mobilenetv2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,286,166 (8.72 MB)

 Trainable params: 28,182 (110.09 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
losses = {
    "type_output": "sparse_categorical_crossentropy",
    "color_output": "sparse_categorical_crossentropy",
    "print_output": "binary_crossentropy",
}

metrics = {
    "type_output": ["accuracy"],
    "color_output": ["accuracy"],
    "print_output": ["accuracy"],
}

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=losses,
    metrics=metrics,
)

EPOCHS_FROZEN = 5  # можно увеличить

history_frozen = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FROZEN,
)


Epoch 1/5
491/491 ━━━━━━━━━━━━━━━━━━━━ 845s 2s/step - color_output_accuracy: 0.4245 - color_output_loss: 1.7686 - loss: 2.8367 - print_output_accuracy: 0.7736 - print_output_loss: 0.4439 - type_output_accuracy: 0.8051 - type_output_loss: 0.6243 - val_color_output_accuracy: 0.6113 - val_color_output_loss: 1.1809 - val_loss: 1.7716 - val_print_output_accuracy: 0.8169 - val_print_output_loss: 0.3876 - val_type_output_accuracy: 0.9330 - val_type_output_loss: 0.2029
Epoch 2/5
491/491 ━━━━━━━━━━━━━━━━━━━━ 835s 2s/step - color_output_accuracy: 0.6002 - color_output_loss: 1.2204 - loss: 1.8023 - print_output_accuracy: 0.8214 - print_output_loss: 0.3724 - type_output_accuracy: 0.9280 - type_output_loss: 0.2095 - val_color_output_accuracy: 0.6419 - val_color_output_loss: 1.0993 - val_loss: 1.6613 - val_print_output_accuracy: 0.8263 - val_print_output_loss: 0.3742 - val_type_output_accuracy: 0.9335 - val_type_output_loss: 0.1877
Epoch 3/5
491/491 ━━━━━━━━━━━━━━━━━━━━ 771s 2s/step - color_output_a

In [ ]:
# Разморозим backbone целиком или частично
backbone.trainable = True

# Можно, например, разморозить только верхние N слоёв:
# for layer in backbone.layers[:-30]:
#     layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # меньше lr при fine-tuning
    loss=losses,
    metrics=metrics,
)

EPOCHS_FINETUNE = 5  # тоже можно менять

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FINETUNE,
)  # при необходимости можно дообучить, но точность распознавания типа и цвета одежды после первого этапа обучения уже приемлема


Epoch 1/5
187/491 ━━━━━━━━━━━━━━━━━━━━ 31:48 6s/step - color_output_accuracy: 0.4594 - color_output_loss: 1.8600 - loss: 3.2832 - print_output_accuracy: 0.7811 - print_output_loss: 0.4823 - type_output_accuracy: 0.7474 - type_output_loss: 0.9409

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_clothing(path_to_img: str):
    img = image.load_img(path_to_img, target_size=(IMG_SIZE, IMG_SIZE))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    x = np.expand_dims(x, axis=0)

    type_pred, color_pred, print_pred = model.predict(x)

    type_idx_pred = np.argmax(type_pred[0])
    color_idx_pred = np.argmax(color_pred[0])
    print_flag = (print_pred[0][0] > 0.5)

    type_name = idx2type[type_idx_pred]
    color_name = idx2color[color_idx_pred]
    print_name = "with_print" if print_flag else "no_print"

    return type_name, color_name, print_name

# Пример:
# sample_path = X_val[0]
# predict_clothing(sample_path)


In [ ]:
X_val[0]

'/content/images/10720.jpg'

In [ ]:
sample_path = X_val[0]
predict_clothing(sample_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


('shorts', 'white', 'no_print')

In [ ]:
model.save("clothing_multitask_mobilenetv2.keras")